- 预测2016.11.01-2016.11.14客户流量.结果为非负整数。
- 提供2015.07.01到2016.10.31（除去2015.12.12）的商家数据，用户支付行为数据以及用户浏览行为数据。
- 提供数据的类型统一为string类型，提交预测的类型为整形。文件统一为utf-8编码，没有标题行，并以“,”分隔的csv格式。

### 供参考的特征生成方法

- centrality measures (mostly median) on sales of the same store/weekday/promo during last 3, 6, 12 and 24 months (some spread measures too but they were more prone to overfitting)
- linear model (ridge) predictions for each store on weekday/promo/dayssince2000 on last 3 and 12 months to help the tree model extrapolate
- number of schoolholidays this week, last week and next week
- aggregates by store (avg sales per customer, promo sales ratio, saturday sales ratio, holiday sales ratio)
- daycounters (before, during and after) for things like position within promo cycle, promo2 cycle, summerholidays and refurbishments

External data that were posted to the forum by others (big thanks!):

- maxtemp and precipmm from the weather files by state
- summerholiday start dates


In [10]:
import pandas as pd
import numpy as np

In [11]:
data_col=['user_id','shop_id','time_stamp']

In [24]:
shop_info_col=['shop_id','city_name','location_id','per_pay','score','comment_cnt','shop_level','cate_1_name','cate_2_name','cate_3_name']

In [29]:
shop_info=pd.read_csv('../dataset/shop_info.txt',header=None)
shop_info.columns=shop_info_col

shop_info.head()

,shop_id,city_name,location_id,per_pay,score,comment_cnt,shop_level,cate_1_name,cate_2_name,cate_3_name
0,1,湖州,885,8,4.0,12.0,2,美食,休闲茶饮,饮品/甜点
1,2,哈尔滨,64,19,NaN,NaN,1,超市便利店,超市,NaN
2,3,南昌,774,5,3.0,2.0,0,美食,休闲茶饮,奶茶
3,4,天津,380,18,NaN,NaN,1,超市便利店,超市,NaN
4,5,杭州,263,2,2.0,2.0,0,美食,休闲食品,生鲜水果


In [30]:
#data_view=pd.read_csv('../dataset/user_view.txt',header=None)

#data_view.columns=data_col

In [31]:
data_pay=pd.read_csv('../dataset/user_view.txt',header=None)
data_pay.columns=data_col

data_pay.head(15)

,user_id,shop_id,time_stamp
0,13201967,1197,2016-10-21 18:00:00
1,19461365,1197,2016-06-28 23:00:00
2,15022321,1197,2016-07-16 19:00:00
3,5440872,1197,2016-07-15 07:00:00
4,12594529,1197,2016-08-07 16:00:00


In [33]:
data_pay.time_stamp=pd.to_datetime(data_pay.time_stamp, format='%Y/%m/%d %H:%M:%S')

In [62]:
data_pay['data']=data_pay.time_stamp.dt.date
#data_pay['day']=data_pay.time_stamp.dt.day
#data_pay['dayofweek']=data_pay.time_stamp.dt.dayofweek
#data_pay['month']=data_pay.time_stamp.dt.month

In [63]:
data_pay.head()

,user_id,shop_id,time_stamp,data,day,dayofweek,month
0,13201967,1197,2016-10-21 18:00:00,2016-10-21,21,4,10
1,19461365,1197,2016-06-28 23:00:00,2016-06-28,28,1,6
2,15022321,1197,2016-07-16 19:00:00,2016-07-16,16,5,7
3,5440872,1197,2016-07-15 07:00:00,2016-07-15,15,4,7
4,12594529,1197,2016-08-07 16:00:00,2016-08-07,7,6,8


In [82]:
feature_daypay=pd.DataFrame(data_pay.groupby(['shop_id','data'])['user_id'].count())
feature_daypay.rename(columns={'user_id':'daypay'},inplace=True)
feature_daypay.to_csv('../tempfile/feature_daypay.csv')
feature_daypay=pd.read_csv('../tempfile/feature_daypay.csv')

In [ ]:
# 14天的预测值 
# 分割数据集

#  2016-10-18  2016-10-31
#  2016-10-3  2016-10-17

# 过去一周平均业绩
#  过去一个月平均业绩
# 过去3个月平均业绩
# 过去半年平均业绩
# 过去一年平均业绩
# 去年同期平均业绩

test=feature_daypay[feature_daypay.data>='2016-10-18']

In [83]:
feature_daypay.head()

,shop_id,data,daypay
0,1,2016-06-22,10
1,1,2016-06-23,18
2,1,2016-06-24,5
3,1,2016-06-25,22
4,1,2016-06-26,15
